<a href="https://colab.research.google.com/github/gokhanturer/NER_Model_SparkNLP/blob/main/BioNLP_09_IOB_to_CoNLL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [6]:
import pandas as pd
import numpy as np
pd.set_option("max_rows",100)

## Getting Train Data

In [25]:
!wget -q https://raw.githubusercontent.com/cambridgeltl/MTL-Bioinformatics-2016/master/data/BioNLP09-IOB/train.tsv
!wget -q https://raw.githubusercontent.com/cambridgeltl/MTL-Bioinformatics-2016/master/data/BioNLP09-IOB/test.tsv

In [27]:
with open("test.tsv") as f:
  test_data = f.read()

In [26]:
with open("train.tsv") as f:
  train_data = f.read()

In [28]:
train_data

'Reactive\tO\noxygen\tO\nintermediate\tO\n-\tO\ndependent\tO\nNF\tO\n-\tO\nkappaB\tO\nactivation\tO\nby\tO\ninterleukin\tB-Protein\n-\tI-Protein\n1beta\tI-Protein\nrequires\tO\n5\tB-Protein\n-\tI-Protein\nlipoxygenase\tI-Protein\nor\tO\nNADPH\tO\noxidase\tO\nactivity\tO\n.\tO\n\nWe\tO\npreviously\tO\nreported\tO\nthat\tO\nthe\tO\nrole\tO\nof\tO\nreactive\tO\noxygen\tO\nintermediates\tO\n(\tO\nROIs\tO\n)\tO\nin\tO\nNF\tO\n-\tO\nkappaB\tO\nactivation\tO\nby\tO\nproinflammatory\tO\ncytokines\tO\nwas\tO\ncell\tO\nspecific\tO\n.\tO\n\nHowever\tO\n,\tO\nthe\tO\nsources\tO\nfor\tO\nROIs\tO\nin\tO\nvarious\tO\ncell\tO\ntypes\tO\nare\tO\nyet\tO\nto\tO\nbe\tO\ndetermined\tO\nand\tO\nmight\tO\ninclude\tO\n5\tB-Protein\n-\tI-Protein\nlipoxygenase\tI-Protein\n(\tO\n5\tB-Protein\n-\tI-Protein\nLOX\tI-Protein\n)\tO\nand\tO\nNADPH\tO\noxidase\tO\n.\tO\n\n5\tB-Protein\n-\tI-Protein\nLOX\tI-Protein\nand\tO\n5\tB-Protein\n-\tI-Protein\nLOX\tI-Protein\nactivating\tO\nprotein\tO\n(\tO\nFLAP\tB-Protein\n)\t

In [18]:
test_data

'Daidzein\tO\nand\tO\ngenistein\tO\nglucuronides\tO\nin\tO\nvitro\tO\nare\tO\nweakly\tO\nestrogenic\tO\nand\tO\nactivate\tO\nhuman\tO\nnatural\tO\nkiller\tO\ncells\tO\nat\tO\nnutritionally\tO\nrelevant\tO\nconcentrations\tO\n.\tO\n\nDaidzein\tO\nand\tO\ngenistein\tO\nglucuronides\tO\n(\tO\nDG\tO\nand\tO\nGG\tO\n)\tO\n,\tO\nmajor\tO\nisoflavone\tO\nmetabolites\tO\n,\tO\nmay\tO\nbe\tO\npartly\tO\nresponsible\tO\nfor\tO\nbiological\tO\neffects\tO\nof\tO\nisoflavones\tO\n,\tO\nsuch\tO\nas\tO\nestrogen\tB-Protein\nreceptor\tI-Protein\nbinding\tO\nand\tO\nnatural\tO\nkiller\tO\ncell\tO\n(\tO\nNK\tO\n)\tO\nactivation\tO\nor\tO\ninhibition\tO\n.\tO\n\nDG\tO\nand\tO\nGG\tO\nwere\tO\nsynthesized\tO\nusing\tO\n3\tO\n-\tO\nmethylcholanthrene\tO\n-\tO\ninduced\tO\nrat\tO\nliver\tO\nmicrosomes\tO\n.\tO\n\nThe\tO\nKm\tO\nand\tO\nVmax\tO\nfor\tO\ndaidzein\tO\nand\tO\ngenistein\tO\nwere\tO\n9\tO\n.\tO\n0\tO\nand\tO\n7\tO\n.\tO\n7\tO\nmicromol\tO\n/\tO\nL\tO\n,\tO\nand\tO\n0\tO\n.\tO\n7\tO\nand\tO\n1\tO

### Converting .tsv to .CoNLL

In [19]:
def data_pre_process(data):

  result = []

  for i in data.split("\n"):
    if i == "": # Each sentence ends ""
      result.append("None\tNone") # We add "None" between each sentences
    else:
      result.append(i)
  
  df = pd.DataFrame(result)

  return df

In [20]:
def conll_generator(data, filename):

  data = data_pre_process(data)

  data.to_csv(filename) # We saved the return value of the data_prepare_function as csv file

  df = pd.read_csv(filename, sep = "\t", usecols = [0,1],names = ['token','entity'])

  # We read csv file , add column name and assigned a NaN value to first row 
  df.iloc[0,:] = np.nan
 
  # to get rid of ","
  df.token = df.token.astype("str").str.findall(r'\,(.*)') 
 
  #df.apply(lambda x: x.split(",")[1]).apply(lambda x: ',' if x == '"' else x)
  df.token = df.token.apply(lambda x : x[0] if len(x)>0  else x)
  
  # We assigned the value np.nan to the places that are None
  df["token"]= df["token"].map(lambda x: np.nan if x == 'None' else x)
  df["entity"] = df["entity"].map(lambda x: np.nan if x == 'None' else x)

  # We added two new columns with value "NN" excluding NaN values
  df["pos1"] = df["token"].map(lambda x: np.nan if type(x)== float else 'NN')
  df["pos2"] = df["token"].map(lambda x: np.nan if type(x)== float else 'NN')

  df = df[["token","pos1","pos2","entity"]]
  
   # We changed the column names according to the conll format
  df.columns = ["-DOCSTART-", "-X-", "-X-", "O"]

  return df

In [29]:
data = train_data

filename = "train_data.csv"

train_data = conll_generator(data, filename)

In [30]:
train_data.head(20)

,-DOCSTART-,-X-,-X-,O
0,[],NN,NN,NaN
1,Reactive,NN,NN,O
2,oxygen,NN,NN,O
3,intermediate,NN,NN,O
4,-,NN,NN,O
5,dependent,NN,NN,O
6,NF,NN,NN,O
7,-,NN,NN,O
8,kappaB,NN,NN,O
9,activation,NN,NN,O


In [23]:
data = test_data

filename = "test_data.csv"

test_data = conll_generator(data, filename)

In [24]:
test_data.head(20)

,-DOCSTART-,-X-,-X-,O
0,[],NN,NN,NaN
1,Daidzein,NN,NN,O
2,and,NN,NN,O
3,genistein,NN,NN,O
4,glucuronides,NN,NN,O
5,in,NN,NN,O
6,vitro,NN,NN,O
7,are,NN,NN,O
8,weakly,NN,NN,O
9,estrogenic,NN,NN,O
